# Integrating Google Calendar API in Python Projects

![](http://chittagongit.com/download/229209)

- [Google Calendar](https://calendar.google.com)

- [Google Calendar API](https://developers.google.com/calendar/)

- [Google Developers Console](https://console.developers.google.com/)

- [Google Calendar API Scopes](https://developers.google.com/calendar/auth)

- [Google Calendar API Reference](https://developers.google.com/calendar/v3/reference/)

## Installation

```
pip install google-api-python-client
```

## OAuth 2.0 Setup

In [1]:
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

In [2]:
scopes = ['https://www.googleapis.com/auth/calendar']

In [ ]:
flow = InstalledAppFlow.from_client_secrets_file("client_secret.json", scopes=scopes)

In [ ]:
credentials = flow.run_console()

In [3]:
import pickle

In [ ]:
pickle.dump(credentials, open("token.pkl", "wb"))

In [4]:
credentials = pickle.load(open("token.pkl", "rb"))

In [5]:
service = build("calendar", "v3", credentials=credentials)

## Get My Calendars

In [6]:
result = service.calendarList().list().execute()

In [ ]:
result['items'][0]

## Get My Calendar Events

In [8]:
calendar_id = result['items'][0]['id']

In [9]:
result = service.events().list(calendarId=calendar_id, timeZone="Asia/Kolkata").execute()

In [ ]:
result['items'][0]

## Create a New Calandar Event

In [11]:
from datetime import datetime, timedelta

In [12]:
start_time = datetime(2023, 5, 26, 19, 30, 0)
end_time = start_time + timedelta(hours=4)
timezone = 'Asia/Kolkata'

In [13]:
event = {
  'summary': 'IPL Final 2023',
  'location': 'Hyderabad',
  'description': 'MI vs TBD',
  'start': {
    'dateTime': start_time.strftime("%Y-%m-%dT%H:%M:%S"),
    'timeZone': timezone,
  },
  'end': {
    'dateTime': end_time.strftime("%Y-%m-%dT%H:%M:%S"),
    'timeZone': timezone,
  },
  'reminders': {
    'useDefault': False,
    'overrides': [
      {'method': 'email', 'minutes': 24 * 60},
      {'method': 'popup', 'minutes': 10},
    ],
  },
}

In [ ]:
service.events().insert(calendarId=calendar_id, body=event).execute()

## Utility function

```
pip install datefinder
```

In [15]:
import datefinder

In [16]:
matches = datefinder.find_dates("26 may 9 PM")

In [ ]:
list(matches)

In [18]:
def create_event(start_time_str, summary, duration=1, description=None, location=None):
    matches = list(datefinder.find_dates(start_time_str))
    if len(matches):
        start_time = matches[0]
        end_time = start_time + timedelta(hours=duration)
    
    event = {
        'summary': summary,
        'location': location,
        'description': description,
        'start': {
            'dateTime': start_time.strftime("%Y-%m-%dT%H:%M:%S"),
            'timeZone': 'Asia/Kolkata',
        },
        'end': {
            'dateTime': end_time.strftime("%Y-%m-%dT%H:%M:%S"),
            'timeZone': 'Asia/Kolkata',
        },
        'reminders': {
            'useDefault': False,
            'overrides': [
                {'method': 'email', 'minutes': 24 * 60},
                {'method': 'popup', 'minutes': 10},
            ],
        },
    }
    return service.events().insert(calendarId='primary', body=event).execute()

In [ ]:
create_event("26 may 9 PM", "Meeting")